In [1]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_tables(conn):
    try:
        cur = conn.cursor()
        cur.execute('''
            CREATE TABLE IF NOT EXISTS location (
                location_id INTEGER PRIMARY KEY AUTOINCREMENT,
                location_abbr TEXT,
                location_desc TEXT,
                geo_location TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS topic_type (
                topic_type_id TEXT PRIMARY KEY,
                topic_type TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS topic (
                topic_id TEXT PRIMARY KEY,
                topic_type_id TEXT,
                topic_desc TEXT,
                FOREIGN KEY (topic_type_id) REFERENCES topic_type (topic_type_id)
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS measure (
                measure_id TEXT PRIMARY KEY,
                measure_desc TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_source (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_source TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS response (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                response TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_footnote (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_value_footnote_symbol TEXT,
                data_value_footnote TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS gender (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                gender TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS race (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                race TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS age (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                age TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS education (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                education TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_value (
                year INTEGER,
                sub_measure_id TEXT,
                data_value_unit TEXT,
                data_value_type TEXT,
                data_value REAL,
                data_value_std_err REAL,
                low_confidence_limit REAL,
                high_confidence_limit REAL,
                sample_size INTEGER,
                stratification_id1 TEXT,
                stratification_id2 TEXT,
                stratification_id3 TEXT,
                stratification_id4 TEXT,
                display_order INTEGER,
                location_id INTEGER,
                topic_type_id TEXT,
                topic_id TEXT,
                measure_id TEXT,
                data_source_id INTEGER,
                response_id INTEGER,
                data_footnote_id INTEGER,
                gender_id INTEGER,
                race_id INTEGER,
                age_id INTEGER,
                education_id INTEGER,
                FOREIGN KEY (location_id) REFERENCES location(location_id),
                FOREIGN KEY (topic_type_id) REFERENCES topic_type(topic_type_id),
                FOREIGN KEY (topic_id) REFERENCES topic(topic_id),
                FOREIGN KEY (measure_id) REFERENCES measure(measure_id),
                FOREIGN KEY (data_source_id) REFERENCES data_source(id),
                FOREIGN KEY (response_id) REFERENCES response(id),
                FOREIGN KEY (data_footnote_id) REFERENCES data_footnote(id),
                FOREIGN KEY (gender_id) REFERENCES gender(id),
                FOREIGN KEY (race_id) REFERENCES race(id),
                FOREIGN KEY (age_id) REFERENCES age(id),
                FOREIGN KEY (education_id) REFERENCES education(id)
            )
        ''')
        conn.commit()
    except sqlite3.Error as e:
        print(e)

In [2]:
import sqlite3
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.base import BaseEstimator, TransformerMixin
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import csv

def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_tables(conn):
    try:
        cur = conn.cursor()
        cur.execute('''
            CREATE TABLE IF NOT EXISTS location (
                location_id INTEGER PRIMARY KEY AUTOINCREMENT,
                location_abbr TEXT,
                location_desc TEXT,
                geo_location TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS topic_type (
                topic_type_id TEXT PRIMARY KEY,
                topic_type TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS topic (
                topic_id TEXT PRIMARY KEY,
                topic_type_id TEXT,
                topic_desc TEXT,
                FOREIGN KEY (topic_type_id) REFERENCES topic_type (topic_type_id)
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS measure (
                measure_id TEXT PRIMARY KEY,
                measure_desc TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_source (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_source TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS response (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                response TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_footnote (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                data_value_footnote_symbol TEXT,
                data_value_footnote TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS gender (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                gender TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS race (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                race TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS age (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                age TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS education (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                education TEXT
            )
        ''')
        cur.execute('''
            CREATE TABLE IF NOT EXISTS data_value (
                year INTEGER,
                sub_measure_id TEXT,
                data_value_unit TEXT,
                data_value_type TEXT,
                data_value REAL,
                data_value_std_err REAL,
                low_confidence_limit REAL,
                high_confidence_limit REAL,
                sample_size INTEGER,
                stratification_id1 TEXT,
                stratification_id2 TEXT,
                stratification_id3 TEXT,
                stratification_id4 TEXT,
                display_order INTEGER,
                location_id INTEGER,
                topic_type_id TEXT,
                topic_id TEXT,
                measure_id TEXT,
                data_source_id INTEGER,
                response_id INTEGER,
                data_footnote_id INTEGER,
                gender_id INTEGER,
                race_id INTEGER,
                age_id INTEGER,
                education_id INTEGER,
                FOREIGN KEY (location_id) REFERENCES location(location_id),
                FOREIGN KEY (topic_type_id) REFERENCES topic_type(topic_type_id),
                FOREIGN KEY (topic_id) REFERENCES topic(topic_id),
                FOREIGN KEY (measure_id) REFERENCES measure(measure_id),
                FOREIGN KEY (data_source_id) REFERENCES data_source(id),
                FOREIGN KEY (response_id) REFERENCES response(id),
                FOREIGN KEY (data_footnote_id) REFERENCES data_footnote(id),
                FOREIGN KEY (gender_id) REFERENCES gender(id),
                FOREIGN KEY (race_id) REFERENCES race(id),
                FOREIGN KEY (age_id) REFERENCES age(id),
                FOREIGN KEY (education_id) REFERENCES education(id)
            )
        ''')
        conn.commit()
    except sqlite3.Error as e:
        print(e)

In [3]:
def load_data_from_db(db_file):
    conn = create_connection(db_file)
    if conn is not None:
        try:
            query = """
            SELECT dv.YEAR as year, dv.Data_Value as data_value, loc.location_desc, tt.topic_type, t.topic_desc,
                   m.measure_desc, ds.data_source, r.Response as response, df.Data_Value_Footnote as data_value_footnote,
                   g.Gender as gender, ra.Race as race, a.Age as age, e.Education as education
            FROM data_value AS dv
            JOIN location AS loc ON dv.location_id = loc.location_id
            JOIN topic_type AS tt ON dv.topic_type_id = tt.topic_type_id
            JOIN topic AS t ON dv.topic_id = t.topic_id
            JOIN measure AS m ON dv.measure_id = m.measure_id
            JOIN data_source AS ds ON dv.data_source_id = ds.id
            JOIN response AS r ON dv.response_id = r.id
            JOIN data_footnote AS df ON dv.data_footnote_id = df.id
            JOIN gender AS g ON dv.gender_id = g.id
            JOIN race AS ra ON dv.race_id = ra.id
            JOIN age AS a ON dv.age_id = a.id
            JOIN education AS e ON dv.education_id = e.id
            """
            df = pd.read_sql(query, conn)
            conn.close()
            return df
        except sqlite3.Error as e:
            print(f"Error executing query: {e}")
            return pd.DataFrame()
    else:
        return pd.DataFrame()
    # Load data

db_file = 'tobacco_use_data.db'  # Specify your SQLite database file path
data = load_data_from_db(db_file)


In [4]:

imputer = SimpleImputer(strategy='median')
data['data_value'] = imputer.fit_transform(data[['data_value']])

train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [5]:
from ydata_profiling import ProfileReport

profile = ProfileReport(train_data, title="Pandas Profiling Report", explorative=True)
profile.to_file("GroupProject.html")


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/anaconda3/lib/python3.11/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'All Ages'')
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/opt/anaconda3/lib/python3.11/site-packages/ydata_profiling/model/missing.py:78: UserWarning: There was an attempt to generate the Heatmap missing values diagrams, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [6]:
categorical_cols = ['location_desc', 'topic_type', 'topic_desc', 'measure_desc', 'data_source', 'response', 'data_value_footnote', 'gender', 'race', 'age', 'education']
numerical_cols = ['year', 'data_value']

# Check for null values
print(train_data.isnull().sum())

train_data[categorical_cols] = train_data[categorical_cols].astype(str)
train_data[numerical_cols] = train_data[numerical_cols].apply(pd.to_numeric, errors='coerce')


year                       0
data_value                 0
location_desc              0
topic_type                 0
topic_desc                 0
measure_desc               0
data_source                0
response               22694
data_value_footnote     1761
gender                     0
race                       0
age                        0
education                  0
dtype: int64


In [7]:
import matplotlib
matplotlib.use('WebAgg') 
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split

def load_data():
    
    data = pd.DataFrame({
        'location_desc': ['Location1', 'Location2', 'Location1', 'Location2'],
        'topic_type': ['Type1', 'Type2', 'Type1', 'Type2'],
        'year': [2001, 2002, 2001, 2002],
        'data_value': [10, None, 20, 30]
    })
    return data

data = load_data()

data['data_value'].fillna(data['data_value'].median(), inplace=True)

# Splitting the data into train and test sets
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

categorical_cols = ['location_desc', 'topic_type']
numerical_cols = ['year', 'data_value']

clean_numerical_data = train_data[numerical_cols].dropna().loc[:, (train_data[numerical_cols].var() != 0)]

if not clean_numerical_data.empty and clean_numerical_data.shape[1] > 1:
    correlation_matrix = clean_numerical_data.corr()
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
    plt.title('Numerical Feature Correlation Heatmap')
    plt.show()
else:
    print("Insufficient data for a meaningful correlation heatmap.")

if 'data_value' in train_data.columns and all(col in train_data.columns for col in categorical_cols):
    for col in categorical_cols:
        if train_data[col].nunique() < 20: 
            plt.figure(figsize=(10, 6))  
            sns.violinplot(x=col, y='data_value', data=train_data)
            plt.title(f'Violin Plot for {col}')
            plt.xticks(rotation=45)
            plt.show()
        else:
            print(f"Skipping {col} due to too many unique categories ({train_data[col].nunique()}).")
else:
    missing_cols = [col for col in categorical_cols if col not in train_data.columns]
    print("Some categorical columns or 'data_value' column are missing:", missing_cols)


/var/folders/r5/_bzgrbtj6tg33lk21qcyg0l80000gn/T/ipykernel_50697/2648839054.py:2: MatplotlibDeprecationWarning: Auto-close()ing of figures upon backend switching is deprecated since 3.8 and will be removed two minor releases later.  To suppress this warning, explicitly call plt.close('all') first.
  matplotlib.use('WebAgg')
/var/folders/r5/_bzgrbtj6tg33lk21qcyg0l80000gn/T/ipykernel_50697/2648839054.py:20: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['data_value'].fillna(data['data_value'].median(), inplace=True)


In [8]:
class DataPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, categorical_cols, numerical_cols):
        self.categorical_cols = categorical_cols
        self.numerical_cols = numerical_cols
        self.imputer_num = SimpleImputer(strategy='mean')
        self.imputer_cat = SimpleImputer(strategy='most_frequent')
        self.encoder = OneHotEncoder(handle_unknown='ignore')
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.imputer_num.fit(X[self.numerical_cols])
        self.imputer_cat.fit(X[self.categorical_cols])
       


In [9]:
import mlflow
import mlflow.sklearn
from dagshub import dagshub_logger

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
import pandas as pd


if not data.empty:
    
    X = data.drop(columns=["data_value"])
    y = data["data_value"]

    
    if y.isnull().any():
        print("Warning: NaN values found in target variable. Removing these records.")
        
        combined = pd.concat([X, y], axis=1)
        clean_combined = combined.dropna(subset=["data_value"])
        X = clean_combined.drop(columns=["data_value"])
        y = clean_combined["data_value"]
    else:
        print("No NaN values in the target variable.")

    
    categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()

    preprocess = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_cols),
            ('cat', Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore'))
            ]), categorical_cols)
        ], remainder='passthrough')

    X[categorical_cols] = X[categorical_cols].astype(str)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model_pipeline = Pipeline([
        ('preprocessor', preprocess),
        ('regressor', LinearRegression())
    ])

    model_pipeline.fit(X_train, y_train)

    y_pred = model_pipeline.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    print(f"MAE: {mae}")
    print(f"MSE: {mse}")
    print(f"RMSE: {rmse}")
else:
    print("Data not loaded or empty.")



No NaN values in the target variable.
MAE: 10.000000000000004
MSE: 100.00000000000007
RMSE: 10.000000000000004


In [11]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR


In [12]:
def evaluate_model(model, model_name, X_train, X_test, y_train, y_test):
    pipeline = Pipeline([
        ('preprocessor', preprocess),
        ('regressor', model)
    ])
    
    pipeline.fit(X_train, y_train)
    
    y_pred = pipeline.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse ** 0.5
    
    print(f"{model_name} - MAE: {mae}, MSE: {mse}, RMSE: {rmse}")

    return pipeline, mae, mse, rmse

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
gb_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
svr_model = SVR()

evaluate_model(rf_model, "Random Forest Regressor", X_train, X_test, y_train, y_test)
evaluate_model(gb_model, "Gradient Boosting Regressor", X_train, X_test, y_train, y_test)
evaluate_model(svr_model, "Support Vector Regressor", X_train, X_test, y_train, y_test)


Random Forest Regressor - MAE: 5.800000000000004, MSE: 33.64000000000005, RMSE: 5.800000000000004
Gradient Boosting Regressor - MAE: 9.999734386011145, MSE: 99.9946877907737, RMSE: 9.999734386011145
Support Vector Regressor - MAE: 1.958016943853707, MSE: 3.8338303524182105, RMSE: 1.958016943853707


(Pipeline(steps=[('preprocessor',
                  ColumnTransformer(remainder='passthrough',
                                    transformers=[('num', StandardScaler(),
                                                   ['year']),
                                                  ('cat',
                                                   Pipeline(steps=[('imputer',
                                                                    SimpleImputer(fill_value='missing',
                                                                                  strategy='constant')),
                                                                   ('onehot',
                                                                    OneHotEncoder(handle_unknown='ignore'))]),
                                                   ['location_desc',
                                                    'topic_type'])])),
                 ('regressor', SVR())]),
 1.958016943853707,
 3.8338303524182105,
 1.9580169

In [13]:
import mlflow
import dagshub
from mlflow.sklearn import log_model
import os

os.environ['MLFLOW_TRACKING_USERNAME'] = 'lollaadityasrivatsav'
os.environ['MLFLOW_TRACKING_PASSWORD'] = 'Srivatsav@04'

def evaluate_and_log_model(model, model_name, X_train, X_test, y_train, y_test):
    mlflow.set_tracking_uri("https://dagshub.com/lollaadityasrivatsav/Python_Group_Project.mlflow")

    with mlflow.start_run(run_name=model_name):
        
        pipeline, mae, mse, rmse = evaluate_model(model, X_train, X_test, y_train, y_test)

        mlflow.log_params({"model_type": model_name})
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("MSE", mse)
        mlflow.log_metric("RMSE", rmse)
        
        log_model(pipeline, "model")


In [18]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Identify categorical columns (This is just an example; adjust based on your dataset)
categorical_features = ['location_desc', 'topic_type']  # Update this with your actual categorical columns

# Create a column transformer with OneHotEncoder for categorical features
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'  # Leave other types of columns unchanged
)

# Create a pipeline that includes preprocessing and the model
model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=100, random_state=42))
])

# Fit the pipeline on the training data
model_pipeline.fit(X_train, y_train)

# Predict on the test data
predictions = model_pipeline.predict(X_test)

# Calculate metrics
mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 33.64000000000005
